<a href="https://colab.research.google.com/github/sundar-nallalagappan/Coursera_NLP_assignments/blob/main/IMDB_TF_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
print("sairam")

sairam


# Import the needed libraries

In [42]:
from keras.layers.pooling.global_average_pooling1d import GlobalAveragePooling1D
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import tensorflow_datasets as tfds

from keras.layers import Dense, Embedding, Flatten, GlobalAveragePooling1D
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


import pandas as pd
import numpy as np
import os, io

2.12.0


# Load the data from Tensorflow data services

In [43]:
imdb, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [44]:
train, test = imdb['train'], imdb['test']
print(type(train))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


In [45]:
for row in train.take(2):
  print(row)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

# convert the tensor to numpy

In [46]:
training_sentences = []
training_labels    = []

for s, l in train:
  #print(s)
  #print(type(s.numpy()))
  #print(type(s.numpy().decode("utf-8")))
  training_sentences.append(s.numpy().decode("utf-8"))
  training_labels.append(l.numpy())

training_sentences[0], training_labels[0]

("This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
 0)

In [47]:
testing_sentences = []
testing_labels    = []

for s, l in test:
  #print(s)
  #print(type(s.numpy()))
  #print(type(s.numpy().decode("utf-8")))
  testing_sentences.append(s.numpy().decode("utf-8"))
  testing_labels.append(l.numpy())

testing_sentences[0], testing_labels[0]


("There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
 1)

In [48]:
training_sentences_final = np.array(training_sentences)
testing_sentences_final = np.array(testing_sentences)

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

training_sentences[0], training_labels_final

("This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
 array([0, 0, 0, ..., 0, 0, 1]))

# Tokenization

In [49]:
oov_token_ = "<OOV>"
embedding_size = 16
vocab_size = 10000
max_length = 120

In [50]:
tokenization = Tokenizer(num_words=vocab_size, oov_token = oov_token_)
tokenization.fit_on_texts(training_sentences_final)

word_index = tokenization.word_index
print(len(word_index))

88583


In [51]:
sequences = tokenization.texts_to_sequences(training_sentences_final)
print(sequences[0:2])

padded = pad_sequences(sequences, padding="post", truncating="post", maxlen=max_length, )
print(padded[0:2])


[[12, 14, 33, 425, 392, 18, 90, 28, 1, 9, 32, 1366, 3585, 40, 486, 1, 197, 24, 85, 154, 19, 12, 213, 329, 28, 66, 247, 215, 9, 477, 58, 66, 85, 114, 98, 22, 5675, 12, 1322, 643, 767, 12, 18, 7, 33, 400, 8170, 176, 2455, 416, 2, 89, 1231, 137, 69, 146, 52, 2, 1, 7577, 69, 229, 66, 2933, 16, 1, 2904, 1, 1, 1479, 4940, 3, 39, 3900, 117, 1584, 17, 3585, 14, 162, 19, 4, 1231, 917, 7917, 9, 4, 18, 13, 14, 4139, 5, 99, 145, 1214, 11, 242, 683, 13, 48, 24, 100, 38, 12, 7181, 5515, 38, 1366, 1, 50, 401, 11, 98, 1197, 867, 141, 10], [11, 26, 75, 571, 6, 805, 2354, 313, 106, 19, 12, 7, 629, 686, 6, 4, 2219, 5, 181, 584, 64, 1454, 110, 2263, 3, 3951, 21, 2, 1, 3, 258, 41, 4677, 4, 174, 188, 21, 12, 4078, 11, 1578, 2354, 86, 2, 20, 14, 1907, 2, 112, 940, 14, 1811, 1340, 548, 3, 355, 181, 466, 6, 591, 19, 17, 55, 1817, 5, 49, 14, 4044, 96, 40, 136, 11, 972, 11, 201, 26, 1046, 171, 5, 2, 20, 19, 11, 294, 2, 2155, 5, 10, 3, 283, 41, 466, 6, 591, 5, 92, 203, 1, 207, 99, 145, 4382, 16, 230, 332, 11, 248

In [52]:
test_sequences = tokenization.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(test_sequences, padding="post", truncating="post", maxlen=max_length, )
print(test_padded[0:2])

[[  48   24  106   13   95 4066   16  740 5065   10   14  312    5    2
   579  349   16 1847 1257    1   16  668 7666 5531    1  761    6   13
  1026    1    1  425  478    1    4    1  327 3560   20  229    3   15
  5742    3   15 1620   15   99    5    2 3550  100   11  772 1498   12
   252  235   11  217    2  366 6454    3   58   93   11   90  102   11
  1498  177   12  252   36    6 1126    1  674    7 4387    1    4    1
   327    7   36 8300  366    5 1403    1   13   29   60   26    6  867
   178   17    4 1037    5   12  227    3   79    4  345   32  345 5159
     5   10    6 1314 1143    2 5619    1]
 [   4    1  696  784    5    4  178    1 2326    1 8758    2 8246   13
  6079    1   14  500    6 2716    9  110  500    6  374    4 1684    1
  9248   17    4 4902    5 8632   15   33    1   37   25 2661  997    5
    20  229   10   14   33    1  674    6 6507   17  115  276    3  574
  2697    1    1  188 9477  109    5    1  958 2661  106    9 1301    5
     2  114 3711    1

# Build model

In [56]:
model = Sequential([
    Embedding(vocab_size, embedding_size, input_length=max_length),
    Flatten(),
    Dense(6, "relu"),
    Dense(1, "sigmoid")
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_1 (Flatten)         (None, 1920)              0         
                                                                 
 dense_4 (Dense)             (None, 6)                 11526     
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [58]:
model.fit(padded, training_labels_final, epochs=15, verbose=2, validation_data=(test_padded, testing_labels_final))

Epoch 1/15
782/782 - 5s - loss: 0.4973 - accuracy: 0.7442 - val_loss: 0.3785 - val_accuracy: 0.8274 - 5s/epoch - 6ms/step
Epoch 2/15
782/782 - 4s - loss: 0.2404 - accuracy: 0.9076 - val_loss: 0.4097 - val_accuracy: 0.8221 - 4s/epoch - 5ms/step
Epoch 3/15
782/782 - 4s - loss: 0.0970 - accuracy: 0.9756 - val_loss: 0.5010 - val_accuracy: 0.8073 - 4s/epoch - 5ms/step
Epoch 4/15
782/782 - 4s - loss: 0.0291 - accuracy: 0.9959 - val_loss: 0.5931 - val_accuracy: 0.8073 - 4s/epoch - 5ms/step
Epoch 5/15
782/782 - 4s - loss: 0.0111 - accuracy: 0.9987 - val_loss: 0.6505 - val_accuracy: 0.8076 - 4s/epoch - 5ms/step
Epoch 6/15
782/782 - 3s - loss: 0.0053 - accuracy: 0.9993 - val_loss: 0.7196 - val_accuracy: 0.8020 - 3s/epoch - 4ms/step
Epoch 7/15
782/782 - 4s - loss: 0.0026 - accuracy: 0.9997 - val_loss: 0.7835 - val_accuracy: 0.8010 - 4s/epoch - 5ms/step
Epoch 8/15
782/782 - 4s - loss: 7.7691e-04 - accuracy: 1.0000 - val_loss: 0.8335 - val_accuracy: 0.8050 - 4s/epoch - 5ms/step
Epoch 9/15
782/782 -

* Looks to be overfitting

# Visualize Word Embeddings

In [70]:
embed_layer = model.layers[0]
embed_weights = embed_layer.get_weights()[0]
embed_weights.shape

(10000, 16)

You will need to generate two files:

vecs.tsv - contains the vector weights of each word in the vocabulary
meta.tsv - contains the words in the vocabulary

In [72]:
reverse_index_word = tokenization.index_word

In [84]:
out_v = io.open("vecs.tsv", "w", encoding="utf-8")
out_m = io.open("meta.tsv", "w", encoding="utf-8")

for i in range(1, vocab_size):
  word_name      = reverse_index_word.get(i)
  word_embedding = embed_weights[i]

  out_m.write(word_name + "\n")
  out_v.write('\t'.join([str(e) for e in word_embedding]) + '\n')

out_m.close()
out_v.close()


In [87]:
try:
  from google.colab import files
except ImportError:
  print("Import exception")
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>